Machine Learning Nanodegree Capstone Project 
Deepa Mohan

2_ConvNet - In this ipynb, we will build our convolutional neural network model and also try various hyperparameter tuning.

In [8]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [9]:
pickle_file = 'SVHN.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (45000, 32, 32) (45000,)
Validation set (6000, 32, 32) (6000,)
Test set (15000, 32, 32) (15000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [10]:
image_size = 32
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (45000, 32, 32, 1) (45000, 10)
Validation set (6000, 32, 32, 1) (6000, 10)
Test set (15000, 32, 32, 1) (15000, 10)


In [11]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    print(hidden.get_shape())
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    print(hidden.get_shape())
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print(reshape)
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

(16, 16, 16, 16)
(16, 8, 8, 16)
Tensor("Reshape:0", shape=(16, 1024), dtype=float32)
(6000, 16, 16, 16)
(6000, 8, 8, 16)
Tensor("Reshape_1:0", shape=(6000, 1024), dtype=float32)
(15000, 16, 16, 16)
(15000, 8, 8, 16)
Tensor("Reshape_2:0", shape=(15000, 1024), dtype=float32)


In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.915756
Minibatch accuracy: 18.8%
Validation accuracy: 10.1%
Minibatch loss at step 50: 1.954394
Minibatch accuracy: 25.0%
Validation accuracy: 12.5%
Minibatch loss at step 100: 2.077112
Minibatch accuracy: 25.0%
Validation accuracy: 20.9%
Minibatch loss at step 150: 1.928695
Minibatch accuracy: 25.0%
Validation accuracy: 28.3%
Minibatch loss at step 200: 1.611924
Minibatch accuracy: 50.0%
Validation accuracy: 32.5%
Minibatch loss at step 250: 1.432525
Minibatch accuracy: 50.0%
Validation accuracy: 39.1%
Minibatch loss at step 300: 1.148630
Minibatch accuracy: 56.2%
Validation accuracy: 49.0%
Minibatch loss at step 350: 1.280634
Minibatch accuracy: 56.2%
Validation accuracy: 46.7%
Minibatch loss at step 400: 1.287131
Minibatch accuracy: 62.5%
Validation accuracy: 54.7%
Minibatch loss at step 450: 1.094871
Minibatch accuracy: 56.2%
Validation accuracy: 55.0%
Minibatch loss at step 500: 0.636504
Minibatch accuracy: 81.2%
Validation accuracy: 59.5%
M

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.


In [10]:
batch_size = 16
patch_size = 5
depth = 8
num_hidden = 48

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    shape = pool.get_shape().as_list()
    print (shape)
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


[16, 8, 8, 8]
[6000, 8, 8, 8]
[15000, 8, 8, 8]


In [11]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.843406
Minibatch accuracy: 6.2%
Validation accuracy: 9.9%
Minibatch loss at step 100: 2.217491
Minibatch accuracy: 18.8%
Validation accuracy: 20.2%
Minibatch loss at step 200: 1.073497
Minibatch accuracy: 56.2%
Validation accuracy: 47.4%
Minibatch loss at step 300: 0.786682
Minibatch accuracy: 87.5%
Validation accuracy: 59.8%
Minibatch loss at step 400: 0.803633
Minibatch accuracy: 75.0%
Validation accuracy: 67.9%
Minibatch loss at step 500: 0.435889
Minibatch accuracy: 93.8%
Validation accuracy: 65.9%
Minibatch loss at step 600: 0.800159
Minibatch accuracy: 56.2%
Validation accuracy: 71.1%
Minibatch loss at step 700: 0.801068
Minibatch accuracy: 81.2%
Validation accuracy: 72.9%
Minibatch loss at step 800: 0.433019
Minibatch accuracy: 93.8%
Validation accuracy: 74.9%
Minibatch loss at step 900: 0.534385
Minibatch accuracy: 81.2%
Validation accuracy: 75.7%
Minibatch loss at step 1000: 0.869546
Minibatch accuracy: 68.8%
Validation accuracy: 75.4%
T

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.


In [19]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
  layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # C1 input 32 x 32
    print(data.get_shape())
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    print(bias1.get_shape())
    # S2 input 28 x 28
    pool2 = tf.nn.max_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    print(pool2.get_shape())
    # C3 input 14 x 14
    conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
    bias3 = tf.nn.relu(conv3 + layer2_biases)
    print(bias3.get_shape())
    # S4 input 10 x 10
    pool4 = tf.nn.max_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    print(pool4.get_shape())
    # F6 input 5 x 5
    shape = pool4.get_shape().as_list()
    reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
    print(reshape)
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

(16, 32, 32, 1)
(16, 28, 28, 16)
(16, 14, 14, 16)
(16, 10, 10, 16)
(16, 5, 5, 16)
Tensor("Reshape:0", shape=(16, 400), dtype=float32)
(6000, 32, 32, 1)
(6000, 28, 28, 16)
(6000, 14, 14, 16)
(6000, 10, 10, 16)
(6000, 5, 5, 16)
Tensor("Reshape_1:0", shape=(6000, 400), dtype=float32)
(15000, 32, 32, 1)
(15000, 28, 28, 16)
(15000, 14, 14, 16)
(15000, 10, 10, 16)
(15000, 5, 5, 16)
Tensor("Reshape_2:0", shape=(15000, 400), dtype=float32)


In [11]:
num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.382908
Minibatch accuracy: 0.0%
Validation accuracy: 9.7%
Minibatch loss at step 500: 0.565625
Minibatch accuracy: 87.5%
Validation accuracy: 69.2%
Minibatch loss at step 1000: 1.128929
Minibatch accuracy: 68.8%
Validation accuracy: 77.9%
Minibatch loss at step 1500: 0.372710
Minibatch accuracy: 93.8%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 0.143789
Minibatch accuracy: 100.0%
Validation accuracy: 82.7%
Minibatch loss at step 2500: 0.498106
Minibatch accuracy: 93.8%
Validation accuracy: 83.3%
Minibatch loss at step 3000: 0.479367
Minibatch accuracy: 87.5%
Validation accuracy: 84.5%
Minibatch loss at step 3500: 0.243918
Minibatch accuracy: 87.5%
Validation accuracy: 82.9%
Minibatch loss at step 4000: 0.315654
Minibatch accuracy: 87.5%
Validation accuracy: 85.3%
Minibatch loss at step 4500: 0.058968
Minibatch accuracy: 100.0%
Validation accuracy: 84.9%
Minibatch loss at step 5000: 0.241110
Minibatch accuracy: 93.8%
Validation accurac

Add a drop-out layer

In [12]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
drop_out = 0.85

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
  layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, keep_prob):
    # C1 input 32 x 32
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    # S2 input 28 x 28
    pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # C3 input 14 x 14
    conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
    bias3 = tf.nn.relu(conv3 + layer2_biases)
    # S4 input 10 x 10
    pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # F5 input 5 x 5
    shape = pool4.get_shape().as_list()
    reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden5 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    # F6
    drop5 = tf.nn.dropout(hidden5, keep_prob)
    hidden6 = tf.nn.relu(tf.matmul(hidden5, layer4_weights) + layer4_biases)
    drop6 = tf.nn.dropout(hidden6, keep_prob)
    return tf.matmul(drop6, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset, drop_out)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.85, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss, global_step=global_step)
  #optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

In [ ]:
num_steps = 10001
 
with tf.Session(graph=graph) as session:
# This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: we initialize our weights and biases
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    #We evaluate our training batch data and labels offset with the help of num_steps and batch size
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    #Compute Loss and output predictions
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
    #Compute Loss, Training, Validation accuracies for each step
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  #Final test accuracy
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.348217
Minibatch accuracy: 12.5%
Validation accuracy: 9.9%
Minibatch loss at step 500: 0.689422
Minibatch accuracy: 75.0%
Validation accuracy: 70.0%
Minibatch loss at step 1000: 0.920920
Minibatch accuracy: 68.8%
Validation accuracy: 75.7%
Minibatch loss at step 1500: 0.602529
Minibatch accuracy: 81.2%
Validation accuracy: 79.3%
Minibatch loss at step 2000: 0.180901
Minibatch accuracy: 93.8%
Validation accuracy: 83.4%
Minibatch loss at step 2500: 0.618765
Minibatch accuracy: 93.8%
Validation accuracy: 83.6%
Minibatch loss at step 3000: 0.433607
Minibatch accuracy: 81.2%
Validation accuracy: 83.0%
Minibatch loss at step 3500: 0.450887
Minibatch accuracy: 87.5%
Validation accuracy: 84.8%
Minibatch loss at step 4000: 0.202680
Minibatch accuracy: 87.5%
Validation accuracy: 86.2%
Minibatch loss at step 4500: 0.018443
Minibatch accuracy: 100.0%
Validation accuracy: 86.5%
Minibatch loss at step 5000: 0.143121
Minibatch accuracy: 93.8%
Validation accurac